# Hominid dataset analysis

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

In [3]:
#read data
hominid = pd.read_csv("Homininos_DataSet (1).csv")
print(hominid.head())

                       Genus_&_Specie        Time Location            Zone  \
0          Australopithecus Anamensis   4.2- 3.9    Africa        oriental   
1  hominino Sahelanthropus tchadensis           7   Africa         central   
2          Australopithecus Afarensis  3.9 - 2.9    Africa        oriental   
3  hominino Sahelanthropus tchadensis           7   Africa         central   
4                Homo Heidelbergensis     500-250   Europa  central-europe   

    Current_Country         Habitat  Cranial_Capacity  Height Incisor_Size  \
0             Kenya  forest-gallery            281.19  132.33          big   
1  Republic of Chad           mixed             94.12   87.49        small   
2             Kenya           mixed            311.48  120.44          big   
3  Republic of Chad           mixed            196.00   88.40        small   
4           Germany     cold forest           1232.46  167.60        small   

  Jaw_Shape  ...            biped        Arms     Foots       

In [4]:
hominid.Time.describe()


count           720
unique           24
top       4.2- 3.9 
freq             30
Name: Time, dtype: object

The column 'Time' is crucial for my analyses but the formatting is not consistent among all data points and cannot be interpreted as an ordered categorical variable. In the following lines I develop a loop in which I

1.Go through each row

2.Check if the line contains a '-'

3.If the line contains a '-', split the line and keep only the first number

4.Check if the number contains a 'ma'

5.If the number contains a 'ma', keep only the first character

6.If the number doesn't contain a 'ma', keep only the first chacter and add a '0.' before it

The previous part of the loop ensures I only keep the oldest known appearance of the species, expressed as million years ago, and converts the ones expressed in thousand years ago to million years ago.


In [13]:
#Converting Time into Oldest known record (mya)

h_time = hominid['Time']
mya_time = []

for item in h_time:
    if '-' in item:
        x = item.split('-')
        if 'ma' in x[0]:
            mya_time.append(x[0][0])
        elif '.' in x[0]:
            mya_time.append(x[0])
        else:
            mya_time.append('0.' + x[0])
    else:
        mya_time.append(item)
        
print(mya_time)

1
['4.2', '7', '3.9 ', '7', '0.500', '2.3', '1.3', '0.300', '1.8', '1.9', '1.9', '2.5', '0.300', '2.8', '2.8', '2.5', '0.100', '2.5', '2.5', '2.5', '1', '1.3', '0.230', '4', '0.230', '0.100', '1.8', '3.5', '4', '4.2', '2.3', '1.8', '1.9', '1.9', '1.8', '2.3', '0.100', '3.5', '1.9', '1.9', '1.9', '2.5', '3.5', '4.2', '2.6', '0.230', '0.500', '6', '3.5', '4', '7', '0.500', '0.600', '3.5', '7', '6', '7', '0.230', '4.2', '6', '3.5', '0.300', '6', '4.2', '2.6', '1.8', '0.230', '2.3', '1.9', '2.3', '4.2', '2.6', '0.100', '1.8', '3.9 ', '0.100', '0.100', '0.300', '3.9 ', '2.5', '4.2', '1.9', '2.3', '4', '0.300', '2.5', '4.2', '6', '0.600', '0.300', '3.5', '2.5', '2.5', '1.8', '6', '2.3', '0.300', '2.5', '3.9 ', '0.500', '1.3', '3.5', '1.8', '0.600', '0.300', '3.5', '0.500', '0.100', '1.9', '2.3', '2.8', '3.5', '1.3', '0.300', '6', '7', '6', '2.8', '2.6', '0.300', '7', '1.8', '2.5', '0.500', '0.300', '0.300', '1.8', '2.5', '0.100', '0.300', '3.5', '0.300', '6', '0.100', '1.3', '1.3', '0.300', 